# Locating longitudinal-transverse correlation in transverse phase space 

The goal of this notebook is to identify the "width" of the observed longitudinal-transverse correlation in the phase space distribution of the BTF bunch at the first emittance station. My idea is to look at the energy distribution of particles within a boundary in 4D transverse phase space. The easiest boundary to implement would be a cube, but it would be nice to reduce things down to one variable. We could try going to normalized coordinates and plotting energy distribution for the particles within a sphere of radius $r$, for example. Or we could the 4D density contours in the transverse phase space, plotting energy distribution vs. contour level.

In [ ]:
import sys
import os
from os.path import join
import importlib
import numpy as np
import pandas as pd
import h5py
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import seaborn as sns
from matplotlib import pyplot as plt
from ipywidgets import interactive
from ipywidgets import widgets
import proplot as pplt

sys.path.append('../..')
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

## Load data 

In [ ]:
folder = '_saved/2021-12-03-VS06'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
info

In [ ]:
filename = info['filename']
coords = utils.load_stacked_arrays(join(folder, f'coords_{filename}.npz'))
shape = tuple([len(c) for c in coords])
print('shape:', shape)

In [ ]:
f = np.memmap(join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

In [ ]:
f_max = np.max(f)
f_min = np.min(f)
if f_min < 0.0:
    print(f'min(f) = {f_min}')
    print('Clipping to zero.')
    f = np.clip(f, 0.0, None)

In [ ]:
dims = ["x", "xp", "y", "yp", "w"]
units = ["mm", "mrad", "mm", "mrad", "MeV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12)

First, look at the energy distribution for integer slices of the other variables.

In [ ]:
sliders = [widgets.IntSlider(min=0, max=len(c)-1, value=len(c)//2) for c in coords[:4]]

In [ ]:
def update(x, xp, y, yp):
    pw = f[x, xp, y, yp, :] / f_max
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(ylim=(0, 1))
    ax.plot(coords[4], pw, color='black')
    plt.show()
    
interactive(update, x=sliders[0], xp=sliders[1], y=sliders[2], yp=sliders[3])

We can also select ranges along each axis.

In [ ]:
range_sliders = [widgets.IntRangeSlider(min=0, max=len(c)-1) for c in coords[:4]]

def update(xr, xpr, yr, ypr):
    
    pw = f[x, xp, y, yp, :] / f_max
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(ylim=(0, 1))
    ax.plot(coords[4], pw, color='black')
    plt.show()
    
interactive(update, x=sliders[0], xp=sliders[1], y=sliders[2], yp=sliders[3])